## Import Dependencies and Open Instrument Handles

In [1]:
import sys
import os
import time
import pyvisa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from binascii import unhexlify
import serial

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"\\Drivers")

from Driver_TECSource_5240 import *
from Driver_PM_PM5020 import *
from Driver_OSA_MS9740B import *
from Driver_LaserSource_71440108 import *

In [3]:
rm = pyvisa.ResourceManager()

tec = TECSource_5240()
tec.open(rm,'ASRL5::INSTR')

pm = PM_PM5020()
pm.open(rm)

osa = OSA_MS9740B()
osa.open(rm)

ld = LaserSource_71440108()
ld.open(rm)

OSA Attenuation Status:  ON



## Initialization

In [4]:
ld.set_output_off(2)
ld.set_output_off(2)
ld.set_output_off(3)
ld.set_output_on(4)

In [6]:
T_Start = 23
T_Stop = 55
T_Step = 0.2
t_wait = 5
n = int((T_Stop-T_Start)/T_Step+1)
pm.set_units(1,"W")

wavelength_start = 1565
wavelength_stop = 1585
wavelength_points = 2001
OSA_Resolution = 0.03
OSA_VBW = 1000
timeout = 10000

Light_engine = "22-49-001"
Channel = 4

Bias_current_mA = 50
osa.set_timeout(20000)

## Function Definition

In [10]:
def sweepOSA(osa, wavelength_start, wavelength_stop, wavelength_points, OSA_Resolution, OSA_VBW):
    osa.set_wavelength(wavelength_start, wavelength_stop, wavelength_points)
    osa.set_resolution_VBW(OSA_Resolution,OSA_VBW)
    osa.sweep_single()
    Power_list = osa.get_sweep_result()
    Power_list_uW = [None] * len(Power_list)
    for i in range(0,len(Power_list)):
        Power_list_uW[i] = 10**(float(Power_list[i])/10)*1000 
    Wavelength_list = np.linspace(wavelength_start,wavelength_stop,wavelength_points)
    Wavelength_peak_nm, Power_peak_dBm = osa.get_peak()   
    if Power_peak_dBm >= -30:
        SMSR_Linewidth_nm, SMSR_dB = osa.get_SMSR()
        Linewidth_3db_nm = osa.get_linewidth(3)
        Linewidth_20db_nm = osa.get_linewidth(20)
    else:
        SMSR_Linewidth_nm, SMSR_dB = 0, 0
        Linewidth_3db_nm = 0
        Linewidth_20db_nm = 0
    return  Wavelength_list, Power_list, Power_list_uW, Wavelength_peak_nm, Power_peak_dBm, SMSR_Linewidth_nm, SMSR_dB, Linewidth_3db_nm, Linewidth_20db_nm

In [11]:
def sweepOSA_Tx(osa, wavelength_start, wavelength_stop, wavelength_points, OSA_Resolution, OSA_VBW):
    osa.set_wavelength(wavelength_start, wavelength_stop, wavelength_points)
    osa.set_resolution_VBW(OSA_Resolution,OSA_VBW)
    osa.sweep_single()
    Power_list = osa.get_sweep_result()
    Power_list_uW = [None] * len(Power_list)
    for i in range(0,len(Power_list)):
        Power_list_uW[i] = 10**(float(Power_list[i])/10)*1000 
    Wavelength_list = np.linspace(wavelength_start,wavelength_stop,wavelength_points)
    Wavelength_peak_1_nm, Power_peak_1_dBm, Wavelength_peak_2_nm, Power_peak_2_dBm, Wavelength_peak_3_nm, Power_peak_3_dBm = osa.get_3_peaks()   
    if Power_peak_1_dBm >= -30:
        SMSR_Linewidth_nm, SMSR_dB = osa.get_SMSR()
        Linewidth_3db_nm = osa.get_linewidth(3)
        Linewidth_20db_nm = osa.get_linewidth(20)
    else:
        SMSR_Linewidth_nm, SMSR_dB = 0, 0
        Linewidth_3db_nm = 0
        Linewidth_20db_nm = 0
    return  Wavelength_list, Power_list, Power_list_uW, Wavelength_peak_1_nm, Power_peak_1_dBm, Wavelength_peak_2_nm, Power_peak_2_dBm, Wavelength_peak_3_nm, Power_peak_3_dBm, SMSR_Linewidth_nm, SMSR_dB, Linewidth_3db_nm, Linewidth_20db_nm

## Measurement Routine (Stead State)

In [47]:
#meas_df = pd.DataFrame()
#t0 = time.time()
for k in range(0,60,1):
    time.sleep(1)
    Voltage_V = ld.get_laser_voltage(Channel)
    T_read = tec.get_temperature()
    t_meas = time.time()-t0
    Wavelength_list, Power_list, Power_list_uW, Wavelength_peak_1_nm, Power_peak_1_dBm, Wavelength_peak_2_nm, Power_peak_2_dBm, Wavelength_peak_3_nm, Power_peak_3_dBm, SMSR_Linewidth_nm, SMSR_dB, Linewidth_3db_nm, Linewidth_20db_nm = sweepOSA_Tx(osa, wavelength_start, wavelength_stop, wavelength_points, OSA_Resolution, OSA_VBW)
    meas_df_temp = pd.DataFrame({'Time':t_meas, 'Light_engine':Light_engine, 'Channel':Channel, 'Bias_current_mA':Bias_current_mA, 'Voltage_V':Voltage_V, 'Temperature_C':T_read, 'Wavelength_nm':Wavelength_list, 'Power_dBm':Power_list, 
                            'Power_uW':Power_list_uW, 'Wavelength_peak_1_nm':Wavelength_peak_1_nm, 'Power_peak_1_nm':Power_peak_1_dBm, 'Wavelength_peak_2_nm':Wavelength_peak_2_nm, 'Power_peak_2_nm':Power_peak_2_dBm, 
                             'Wavelength_peak_3_nm':Wavelength_peak_3_nm, 'Power_peak_3_nm':Power_peak_3_dBm, 'SMSR_dB':SMSR_dB, 'SMSR_linewidth_nm':SMSR_Linewidth_nm,
                            '3dB_linewidth_nm':Linewidth_3db_nm, '20dB_linewidth_nm':Linewidth_20db_nm})
    data = [meas_df_temp, meas_df]
    meas_df = pd.concat(data,ignore_index=True,sort=False)

## Save and Plot Data

In [48]:
meas_df.to_csv(r"Spectrum_TX_Temp_Sweep2.csv")

In [ ]:
plt.plot(meas_df.Time,meas_df.Power_peak_1_dBm)

## Measurement Routine (Temperature Ramp)

In [ ]:
meas_df = pd.DataFrame()
t0 = time.time()

In [ ]:
for k in range(0,n+1,1):
    if(T_Start+k*T_Step>47):
        T_Set = 47
    else:
        T_Set = T_Start+k*T_Step
    tec.set_temperature(T_Set)
    time.sleep(t_wait)
    Voltage_V = ld.get_laser_voltage(Channel)
    T_read = tec.get_temperature()
    t_meas = time.time()-t0
    Wavelength_list, Power_list, Power_list_uW, Wavelength_peak_1_nm, Power_peak_1_dBm, Wavelength_peak_2_nm, Power_peak_2_dBm, Wavelength_peak_3_nm, Power_peak_3_dBm, SMSR_Linewidth_nm, SMSR_dB, Linewidth_3db_nm, Linewidth_20db_nm = sweepOSA_Tx(osa, wavelength_start, wavelength_stop, wavelength_points, OSA_Resolution, OSA_VBW)
    meas_df_temp = pd.DataFrame({'Time':t_meas, 'Light_engine':Light_engine, 'Channel':Channel, 'Bias_current_mA':Bias_current_mA, 'Voltage_V':Voltage_V, 'Temperature_C':T_read, 'Wavelength_nm':Wavelength_list, 'Power_dBm':Power_list, 
                            'Power_uW':Power_list_uW, 'Wavelength_peak_1_nm':Wavelength_peak_1_nm, 'Power_peak_1_nm':Power_peak_1_dBm, 'Wavelength_peak_2_nm':Wavelength_peak_2_nm, 'Power_peak_2_nm':Power_peak_2_dBm, 
                             'Wavelength_peak_3_nm':Wavelength_peak_3_nm, 'Power_peak_3_nm':Power_peak_3_dBm, 'SMSR_dB':SMSR_dB, 'SMSR_linewidth_nm':SMSR_Linewidth_nm,
                            '3dB_linewidth_nm':Linewidth_3db_nm, '20dB_linewidth_nm':Linewidth_20db_nm})
    data = [meas_df_temp, meas_df]
    meas_df = pd.concat(data,ignore_index=True,sort=False)

## View, Plot and Save Data

In [44]:
meas_df

,Time,Light_engine,Channel,Bias_current_mA,Voltage_V,Temperature_C,Wavelength_nm,Power_dBm,Power_uW,Wavelength_peak_1_nm,Power_peak_1_nm,Wavelength_peak_2_nm,Power_peak_2_nm,Wavelength_peak_3_nm,Power_peak_3_nm,SMSR_dB,SMSR_linewidth_nm,3dB_linewidth_nm,20dB_linewidth_nm
0,1118.765702,22-49-001,4,50,1.013,49.71,1565.00,-67.28,1.870682e-04,1574.86,-6.75,1574.86,-6.75,1578.40,-7.51,0.16,3.54,6.028,6.028
1,1118.765702,22-49-001,4,50,1.013,49.71,1565.01,-59.04,1.247384e-03,1574.86,-6.75,1574.86,-6.75,1578.40,-7.51,0.16,3.54,6.028,6.028
2,1118.765702,22-49-001,4,50,1.013,49.71,1565.02,-56.63,2.172701e-03,1574.86,-6.75,1574.86,-6.75,1578.40,-7.51,0.16,3.54,6.028,6.028
3,1118.765702,22-49-001,4,50,1.013,49.71,1565.03,-63.78,4.187936e-04,1574.86,-6.75,1574.86,-6.75,1578.40,-7.51,0.16,3.54,6.028,6.028
4,1118.765702,22-49-001,4,50,1.013,49.71,1565.04,-69.86,1.032761e-04,1574.86,-6.75,1574.86,-6.75,1578.40,-7.51,0.16,3.54,6.028,6.028
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
388189,1.036517,22-49-001,4,50,1.019,23.00,1584.96,-57.57,1.749847e-03,1572.10,-6.43,1575.61,-6.75,1577.98,-7.24,0.35,3.51,5.964,5.911
388190,1.036517,22-49-001,4,50,1.019,23.00,1584.97,-61.05,7.852356e-04,1572.10,-6.43,1575.61,-6.75,1577.98,-7.24,0.35,3.51,5.964,5.911
388191,1.036517,22-49-001,4,50,1.019,23.00,1584.98,-57.41,1.815516e-03,1572.10,-6.43,1575.61,-6.75,1577.98,-7.24,0.35,3.51,5.964,5.911
388192,1.036517,22-49-001,4,50,1.019,23.00,1584.99,-120.0,1.000000e-09,1572.10,-6.43,1575.61,-6.75,1577.98,-7.24,0.35,3.51,5.964,5.911


In [ ]:
meas_df.to_csv(r"Spectrum_TX_Temp_Sweep.csv")

## Close Instrument Handles

In [5]:
pm.close()
tec.close()
osa.close()
ld.close()